In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
from scipy.spatial import distance_matrix
import scipy.stats as st
import faiss
from tqdm import tqdm


from src.common.scales import scales
from src.common.utils import generate_population, mutation, crossover
from src.common.fitness import (griewank, schwefel, ackley, rastrigin)
from src.elegant_fuzzy_genetic_algorithms.helpers.all_params_wrapper import AllEFGAParamsParallelWrapper
from src.elegant_fuzzy_genetic_algorithms.priority_diff_simulation import simulation_priorities
from src.common.approximation_helpers import (generate_search_space, init_param_index, estimate_by_index)
from src.gendered_selection.faster_fuzzy_logic.generalized_partition_inferrer import GeneralizedInferrer
from src.gendered_selection.age_diff_estimator import Simulation as AgeDiffSimulation
from src.gendered_selection.conf.gendered_selection_config import Config as GenderedSelectionConfig


np.random.seed(1)

Given that 90+% of time spent running fuzzy genetic algorithms is spent on using fuzzy logic, the idea is to make an approximation using nearest neighbors. 

## Priority estimation (EFGA)

### Getting the error confidence interval

In [29]:
priority_inferencer = AllEFGAParamsParallelWrapper(n_terms_params=3, n_terms_priority=7)

In [30]:
c1_range = np.linspace(start=0, stop=1, num=60)
c2_range = np.linspace(start=0, stop=1, num=60)

In [31]:
params_combinations = np.array(np.meshgrid(c1_range, c2_range)).T.reshape(-1, 2)
priorities = priority_inferencer.infer_priority(c1=params_combinations[:, 0], c2=params_combinations[:, 1])

In [32]:
entries = np.random.uniform(0, 1, size=(200, 2))

In [33]:
priorities_est = priorities[np.argmin(distance_matrix(entries, params_combinations), axis=1)]
priorities_actual =  priority_inferencer.infer_priority(c1=entries[:, 0], c2=entries[:, 1])

In [34]:
diff = np.abs(priorities_est - priorities_actual)

In [35]:
conf_int = st.t.interval(alpha=0.95, df=len(diff)-1, loc=np.mean(diff), scale=st.sem(diff)) 

We know the 95% confidence interval of the priority estimation error. If we prove that it's smaller than 95% of differences between the current priority and next best, then usage of this estimation is appropriate. 

### Estimating priority difference

In [49]:
priorities = []

for fn in [griewank, schwefel, rastrigin, ackley]:
    fn_name = fn.__name__
    priorities_ = simulation_priorities(N=100, epochs=200, fitness_fn=fn, population_scale=scales[fn_name][0], 
                      mutation_scale=scales[fn_name][1], seed=1)
    priorities.append(priorities_)

100%|██████████| 200/200 [00:20<00:00,  9.64it/s]


In [50]:
priorities_mtr = np.array(priorities)
priorities_mtr = np.sort(priorities_mtr, axis=1)
priority_difference_mtr = np.diff(priorities_mtr, axis=1)
priority_difference =  priority_difference_mtr.ravel()

In [51]:
conf_int_actual_diff = st.t.interval(alpha=0.95, df=len(priority_difference)-1, loc=np.mean(priority_difference), scale=st.sem(priority_difference)) 

In [52]:
conf_int_actual_diff, conf_int

((0.026625617917013247, 0.027913588650583764),
 (0.005057507916810315, 0.00619624680288484))

In [53]:
first_second_diff = np.abs(np.diff(priority_difference_mtr[:, :2], axis=1).ravel())
conf_int_first_second = st.t.interval(alpha=0.95, df=len(first_second_diff)-1, loc=np.mean(first_second_diff), scale=st.sem(first_second_diff)) 

In [54]:
conf_int_first_second

(0.05903070832699521, 0.09724286275938558)

### Conclusion

Provided that the actual difference between first and second entry, as well as difference is much bigger than actual errror of approximation (even in the worst case the error is 10 times smaller), we can successfully use approximation techniques. 

## Partner age estimation (Gendered selection)

### Getting actual error for age estimation

In [23]:
params_combinations = generate_search_space(n_splits=200, ranges=[(0, 1), (0, 10)])
gi = GeneralizedInferrer(7)

In [24]:
y = [gi.infer_partner_age(*params_combinations[i, :]) for i in tqdm(range(params_combinations.shape[0]))]

100%|██████████| 40000/40000 [03:27<00:00, 192.65it/s]


In [25]:
y = np.array(y)

In [26]:
entries_test = np.hstack([np.random.uniform(0, 1, size=(200, 1)), np.random.uniform(0, 10, size=(200, 1))])
age_est = y[np.argmin(distance_matrix(entries_test, params_combinations), axis=1)]
age_actual = np.array([gi.infer_partner_age(*entries_test[i, :]) for i in tqdm(range(entries_test.shape[0]))])

100%|██████████| 200/200 [00:01<00:00, 114.50it/s]


In [27]:
diff = np.abs(age_est - age_actual)
conf_int_age_error = st.t.interval(alpha=0.95, df=len(diff)-1, loc=np.mean(diff), scale=st.sem(diff)) 
conf_int_age_error

(0.0009237931731446055, 0.00124733269662373)

### Estimating confidence intervals for age difference for females

In [28]:

ages = []

for fn in [griewank, schwefel, rastrigin, ackley]:
    fn_name = fn.__name__
    sim = AgeDiffSimulation(conf=GenderedSelectionConfig(100), fitness_fn=fn, mutation=mutation, crossover=crossover)
    ages_ = sim.run(n_epochs=200, n_partitions=7, population_scale=scales[fn_name][0], mutation_scale=scales[fn_name][1], seed=1)
    ages.append(ages_)

  0%|          | 1/200 [00:00<00:23,  8.34it/s]

[-249.35633071 -246.12872579 -245.56878051 -219.57428921 -219.41105259
 -218.75044051 -213.22218905 -210.92547902 -209.88358201 -201.62965166]


 67%|██████▋   | 134/200 [00:13<00:06, 10.10it/s]


Early stopping at epoch: 134, population died


  0%|          | 1/200 [00:00<00:20,  9.64it/s]

...

[-164.04324218 -142.81374051 -124.27624217 -110.47992893 -109.60358413
 -108.34798061 -108.12620076 -107.1076225  -105.39232329 -101.18524974]


  0%|          | 1/200 [00:00<00:21,  9.34it/s]

[-22.237698   -22.2148927  -22.00886026 -21.99755589 -21.98829155
 -21.97395631 -21.96101749 -21.95221456 -21.94448626 -21.91396373]


 51%|█████     | 102/200 [00:09<00:09, 10.68it/s]

Early stopping at epoch: 102, population died


In [29]:
diffs_age = np.concatenate([np.concatenate([np.diff(np.sort(i)) for i in ages[j]]) for j in range(len(ages))])
conf_int_age_diff = st.t.interval(alpha=0.95, df=len(diffs_age)-1, loc=np.mean(diffs_age), scale=st.sem(diffs_age)) 
conf_int_age_diff

(0.002322242730583462, 0.0025186984783038373)

In [30]:
diffs_age = np.concatenate([np.concatenate([np.diff(np.sort(i)[:2]) for i in ages[j]]) for j in range(len(ages))])
conf_int_age_diff = st.t.interval(alpha=0.95, df=len(diffs_age)-1, loc=np.mean(diffs_age), scale=st.sem(diffs_age)) 
conf_int_age_diff

(0.01107746234098387, 0.023435579298824324)

### Conclusions

Given that upper range of 95% confidence interval for error is smaller than lower range of 95% range of age difference between adjacent female ages, we can conclude that error is small enought to not be noticeable in partner selection. 